![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

filename_features = "/content/features_info.txt"
filename_labels = "/content/activity_labels.txt"

filename_subtrain = "/content/train/subject_train.txt"
filename_xtrain = "/content/train/X_train.txt"
filename_ytrain = "/content/train/y_train.txt"

filename_subtest = "/content/test/subject_test.txt"
ffilename_xtest = "/content/test/X_test.txt"
filename_ytest = "/content/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#").squeeze()
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id']).squeeze()
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id']).squeeze()
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [12]:
%%time
clf = DecisionTreeClassifier(ccp_alpha=0.001).fit(X_train, y_train)

CPU times: user 544 ms, sys: 0 ns, total: 544 ms
Wall time: 542 ms


In [13]:
train_score = clf.score(X_train, y_train)
test_score = clf.score(X_test, y_test)

print(f'Acurácia da melhor árvore na base de treino:    {train_score*100:.1f}')
print(f'Acurácia da melhor árvore na base de teste:     {test_score*100:.1f}')

Acurácia da melhor árvore na base de treino:    93.9
Acurácia da melhor árvore na base de teste:     80.3


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [14]:
%%time
prcomp = PCA(n_components=1).fit(X_train)
pc_train = prcomp.transform(X_train)
pc_test = prcomp.transform(X_test)

CPU times: user 215 ms, sys: 109 ms, total: 325 ms
Wall time: 172 ms


In [15]:
clf = DecisionTreeClassifier(ccp_alpha=0.001).fit(pc_train, y_train)

train_score = clf.score(pc_train, y_train)
test_score = clf.score(pc_test, y_test)

print(f'Acurácia da melhor árvore na base de treino:    {train_score*100:.1f}')
print(f'Acurácia da melhor árvore na base de teste:     {test_score*100:.1f}')

Acurácia da melhor árvore na base de treino:    53.5
Acurácia da melhor árvore na base de teste:     50.5


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [19]:
prcomp_list = []
n_comp = [1, 2, 5, 10, 50]

for n in n_comp:
    start = time.time()

    n_components = min(n, min(X_train.shape))

    prcomp = PCA(n_components=n_components).fit(X_train)
    pc_train = prcomp.transform(X_train)
    pc_test = prcomp.transform(X_test)

    clf = DecisionTreeClassifier(random_state=101, ccp_alpha=0.001).fit(pc_train, y_train)

    train_score = clf.score(pc_train, y_train)
    test_score = clf.score(pc_test, y_test)

    print(f'Acurácia da melhor árvore na base de treino para {n_components} componentes:    {train_score*100:.1f}')
    print(f'Acurácia da melhor árvore na base de teste {n_components} componentes:          {test_score*100:.1f}')
    print(f'Tempo de processamento: {time.time() - start}')
    print('-----')

Acurácia da melhor árvore na base de treino para 1 componentes:    53.5
Acurácia da melhor árvore na base de teste 1 componentes:          50.5
Tempo de processamento: 0.2924387454986572
-----
Acurácia da melhor árvore na base de treino para 2 componentes:    56.9
Acurácia da melhor árvore na base de teste 2 componentes:          52.5
Tempo de processamento: 0.21025395393371582
-----
Acurácia da melhor árvore na base de treino para 5 componentes:    77.3
Acurácia da melhor árvore na base de teste 5 componentes:          71.1
Tempo de processamento: 0.31455469131469727
-----
Acurácia da melhor árvore na base de treino para 10 componentes:    78.8
Acurácia da melhor árvore na base de teste 10 componentes:          72.4
Tempo de processamento: 0.3581068515777588
-----
Acurácia da melhor árvore na base de treino para 49 componentes:    85.3
Acurácia da melhor árvore na base de teste 49 componentes:          73.0
Tempo de processamento: 0.8677287101745605
-----


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

A acurácia na base de treino aumentou constantemente com mais componentes, mas a acurácia na base de teste não melhorou tanto, sugerindo um risco de overfitting

O tempo de processamento aumentou à medida que mais componentes foram usados, resultando em um maior custo computacional para o treinamento do modelo.